In [1]:
import pandas as pd # python에서 데이터를 조작할 때 쓰는 툴
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.externals import joblib

train_df = pd.read_pickle("soma_goods_train.df")
vectorizer = CountVectorizer()

In [2]:
# set d_list, cate_list

d_list = []
cate_list = []
for each in train_df.iterrows():
    cate = ";".join([each[1]['cate1'],each[1]['cate2'],each[1]['cate3']])
    d_list.append(each[1]['name'])
    cate_list.append(cate)

In [3]:
# make cate_dict

cate_dict = dict(zip(list(set(cate_list)),range(len(set(cate_list))))) # 카테고리 별로 id를 만들어주는 코드.
# print cate_dict[u'디지털/가전;네트워크장비;KVM스위치']
# print cate_dict[u'패션의류;남성의류;정장']s

In [4]:
# make y_list

y_list = []
for each in train_df.iterrows(): # iterrows 데이터를 도는 애
    cate = ";".join([each[1]['cate1'],each[1]['cate2'],each[1]['cate3']])
    y_list.append(cate_dict[cate])

In [5]:
# define removeStopWords

mapping = { 'A/S':'AS'}
stop_words = [u"%즉시할인쿠폰",  u"5% 즉시할인", u"0% 즉시할인", u"% 즉시할인", u"5% 할인", u"0% 할인", u"5%할인", u"0%할인", u"% 할인", u"적립",
              u"쿠폰", u"즉시",u"정품", u"증정", u"할인",
              u"해외직배송", u"직배송", u"무료배송", u"안전배송", u"당일", u"배송", u"발송", u"빠른",
              u"신한카드", u"BC카드", u"KB국민카드", u"국민카드", u"롯데카드", u"KB카드", u"현대카드", u"삼성카드", u"일까지"]
symbols = ["[","]","(",")", "+", "-", "/", "?", "~", "*", u"☆", u"★", u"●", u"♥", u"■", u"▶", u"◀", u"↑"]
ambg_words = [u"해외", u"구매대행"]

start = 0
length = 1
remove_ambg = True

# define concatenate function
def concate(list1, list2) :
    for each in list2:
        list1.append(each)
    return list1

remove_list = concate(stop_words, symbols)
if remove_ambg:
    remove_list = concate(remove_list,ambg_words)
    
def removeStopWords(word) :
     # replace
    for before, after in mapping.iteritems():
        word = word.replace(before, after)
    #remove
    for each in remove_list :
        word = word.replace(each," ")
    return word

In [6]:
# define splitMorph using konlpy

from konlpy.tag import Twitter
from konlpy.utils import pprint
kkma = Twitter()

def splitMorph(word) :
    morphs = kkma.pos(word)
    words = ""
    for morph in morphs :
        word = morph[0]
        words += " " + word
    return words

In [7]:
print d_list[10]
print splitMorph(d_list[10])

[해외]IntelIntel Pentium Dual Core G2030 LGA 1155 Processor BX80637G2030
 [ 해외 ] IntelIntel Pentium Dual Core G 2030 LGA 1155 Processor BX 80637 G 2030


In [8]:
# remove all stop words
# And split all morphs

length = len(d_list)
for i in range(0, length):
    d_list[i] = removeStopWords(d_list[i])
    d_list[i] = splitMorph(d_list[i])

In [9]:
# JSON result 미리 읽어오기

list_caffe = []
list_caffe = joblib.load('list_caffe.dat')
length = len(list_caffe)
print length

10000


In [10]:
########### append string

size = 10000
use_caffe = True
if use_caffe :
    for num in range(0, size):
        res = list_caffe[num]['result']
        
        if res[0] :
            for each in res[1] :
                d_list[num] += " " + each[0].replace(" ", "")
            for each in res[2] :
                d_list[num] += " " + each[0].replace(" ", "")

print d_list[1000]
# print train_df[test_num-1:test_num]

 LG 유플러스 번호 이동 갤럭시 A 5 SM A 500 L LTE 음성 무한 자유 124 할부 원금 173500 원 가입 비 분납 유심 비후 청구 위약금 남은 할부 원금 , 받은 지원 금 , 약정 금 부가 서비스 monitor iPod screen website bookjacket computer machine personalcomputer electronicequipment digitalcomputer


In [11]:
######### training set과 test set 나누기
d_list_test = []
d_list_train = []
y_list_test = []
y_list_train = []
size = len(d_list)
for i in range(0,size) :
    if i % 10 == 0:
        d_list_test.append(d_list[i])
        y_list_test.append(y_list[i])
    else :
        d_list_train.append(d_list[i])
        y_list_train.append(y_list[i])
print len(d_list_test)
print len(y_list_train)

1000
9000


In [12]:
##############################################
# save
#joblib.dump(d_list_test, "d_list_test.dat")
#joblib.dump(d_list_train, "d_list_train.dat")
#joblib.dump(y_list_test, "y_list_test.dat")
#joblib.dump(y_list_train, "y_list_train.dat")


In [4]:
#############################################
# load
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.externals import joblib
d_list_test = joblib.load('d_list_test.dat')
d_list_train = joblib.load('d_list_train.dat')
y_list_test = joblib.load('y_list_test.dat')
y_list_train = joblib.load('y_list_train.dat')
vectorizer = CountVectorizer()

In [5]:
# create x_list
x_list = vectorizer.fit_transform(d_list_train)
print x_list.shape

(9000, 16168)


In [6]:
from sklearn.svm import LinearSVC
from sklearn.grid_search import GridSearchCV
import numpy as np
#svc_param = {'C':np.logspace(-2,0,20)} # 엔진이 학습을 할 때 파라미터에 따라서 최적화를 해준다. 속도를 빠르게 하고싶으면 (-2,0,5)로 바꿈.
svc_param = {'C':np.logspace(-2,0,5)} # 엔진이 학습을 할 때 파라미터에 따라서 최적화를 해준다. 속도를 빠르게 하고싶으면 (-2,0,5)로 바꿈.
gs_svc = GridSearchCV(LinearSVC(loss='l2'),svc_param,cv=5,n_jobs=4) # n_jobs 몇개 코어로 돌릴거냐

In [7]:
gs_svc.fit(x_list, y_list_train)

/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/cross_validation.py:516: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)
/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/svm/classes.py:197: DeprecationWarning: loss='l2' has been deprecated in favor of loss='squared_hinge' as of 0.16. Backward compatibility for the loss='l2' will be removed in 1.0
  DeprecationWarning)
/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/svm/classes.py:197: DeprecationWarning: loss='l2' has been deprecated in favor of loss='squared_hinge' as of 0.16. Backward compatibility for the loss='l2' will be removed in 1.0
  DeprecationWarning)
/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/svm/classes.py:197: DeprecationWarning: loss='l2' has been deprecated in favor of loss='squared_hinge' as of 0.16. Backward compatibility f

GridSearchCV(cv=5, error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='l2', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0),
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'C': array([ 0.01   ,  0.03162,  0.1    ,  0.31623,  1.     ])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [8]:
print gs_svc.best_params_, gs_svc.best_score_
# overfitting underfitting 때문에.
clf = LinearSVC(C=gs_svc.best_params_['C'])
# C 값이 overfitting을 결정한다.
clf.fit(x_list,y_list_train)  # fit 하면 학습이 됨.

{'C': 0.10000000000000001} 0.695555555556


LinearSVC(C=0.10000000000000001, class_weight=None, dual=True,
     fit_intercept=True, intercept_scaling=1, loss='squared_hinge',
     max_iter=1000, multi_class='ovr', penalty='l2', random_state=None,
     tol=0.0001, verbose=0)

In [11]:
# test
size = len(d_list_test)

sum = 0
for num in range (0, size):
    title = d_list_test[num]
    vec = vectorizer.transform([title])
    pred = clf.predict(vec)[0]
    answer = y_list_test[num]
    if pred == answer:
        sum = sum + 1
print sum

774


In [12]:
# 만든 모델 저장
test = True
if test:
    joblib.dump(clf,'classify_test.model',compress=3)   # 학습 데이터가 저장되어 있음
    joblib.dump(cate_dict,'cate_dict_test.dat',compress=3)
    joblib.dump(vectorizer,'vectorizer_test.dat',compress=3)
else :
    joblib.dump(clf,'classify.model',compress=3)   # 학습 데이터가 저장되어 있음
    joblib.dump(cate_dict,'cate_dict.dat',compress=3)
    joblib.dump(vectorizer,'vectorizer.dat',compress=3)

NameError: name 'cate_dict' is not defined